1. [Types of agent](https://python.langchain.com/v0.1/docs/modules/agents/agent_types/)
2. [Custom Tools](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)
3. [version_3_custom_tools](https://python.langchain.com/docs/how_to/custom_tools/)

In [6]:
!pip install -qU langchain langsmith langchain-core langchain-community langchain-experimental langchain-openai langchain-groq pypdf langchain-text-splitters langchain-chroma sentence_transformers google-search-results wikipedia duckduckgo-search arxiv langgraph

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.5 MB/s eta 0:00:00


# Load Envs

In [2]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')
os.environ['LANGSMITH_API_KEY'] = userdata.get('LANGSMITH_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

# Setup Langsmith

In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGSMITH_API_KEY')
os.environ["LANGCHAIN_PROJECT"] = "langgraph_learning"

# Initialize LLM

In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = 'llama-3.1-8b-instant')

# Tools

### Prebuilt tools

In [7]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

In [10]:
search.invoke("Who is Kobe Bryant?")

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


"Bryant gave himself the nickname Black Mamba in the mid-2000s, and the epithet became widely adopted by the public. He won gold medals on the 2008 and 2012 U.S.\xa0... Dec 9, 2024 ... Kobe Bryant, American professional basketball player who helped lead the Los Angeles Lakers of the National Basketball Association (NBA) to\xa0... Oct 26, 2023 ... Former pro basketball player Kobe Bryant won five NBA titles with the Los Angeles Lakers while establishing himself as one of the game's all-time greats. Granity Studios - CEO, Writer, Producer. After a 20-year career spent entirely with the Lakers, Kobe Bryant retired as the franchise career leader in points, games, minutes, and steals."

In [48]:
from langchain_community.utilities import ArxivAPIWrapper

arxiv = ArxivAPIWrapper(
    top_k_results = 3,
    ARXIV_MAX_QUERY_LENGTH = 300,
    load_max_docs = 3,
    load_all_available_meta = False,
    doc_content_chars_max = 40000
)

In [40]:
arxiv.run("What is the paper Attention is all you need about?")

'Published: 2011-02-18\nTitle: Quantum Anticipation Explorer\nAuthors: Hans-Rudolf Thomann\nSummary: Quantum anticipation explorer is a computer program allowing the numerical\nexploration of quantum anticipation which has been analyzed in arXiv:0810.183v1\nand arXiv:1003.1090v1 for H-Atom, equidistant, random and custom spectra. This\ntool determines the anticipation strength at those times orthogonal evolution\nis possible. This paper is the user\'s guide explaining its capabilities,\ninstallation and usage, and documenting the mathematics and algorithms\nimplemented in the software. A zip file containing the setup and documentation\ncan be downloaded from\nhttp://www.thomannconsulting.ch/public/aboutus/aboutus-en.htm free of cost.\n\nPublished: 2018-07-02\nTitle: The Matrix Calculus You Need For Deep Learning\nAuthors: Terence Parr, Jeremy Howard\nSummary: This paper is an attempt to explain all the matrix calculus you need in order\nto understand the training of deep neural network

In [31]:
# from langchain.agents import load_tools

# arxiv_tools = load_tools(
#     ["arxiv"],
# )

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [15]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [16]:
wikipedia.run("MS Dhoni")

"Page: MS Dhoni\nSummary: Mahendra Singh Dhoni ( ; born 7 July 1981) is an Indian professional cricketer who plays as a right-handed batter and a wicket-keeper. Widely regarded as one of the most prolific wicket-keeper batsmen and captains, he represented the Indian cricket team and was the captain of the side in limited overs formats from 2007 to 2017 and in test cricket from 2008 to 2014. Dhoni has captained the most international matches and is the most successful Indian captain. He has led India to victory in the 2007 ICC World Twenty20, the 2011 Cricket World Cup, and the 2013 ICC Champions Trophy, being the only captain to win three different limited overs ICC tournaments. He also led the teams that won the Asia Cup in 2010, 2016 and was a member of the title winning squad in 2018.\nBorn in Ranchi, Dhoni made his first class debut for Bihar in 1999. He made his debut for the Indian cricket team on 23 December 2004 in an ODI against Bangladesh and played his first test a year late

In [56]:
tools = [search, wikipedia]

# Create Agent

In [57]:
from langchain import hub

# Get the prompt to use - you can modify this!
# prompt = hub.pull("hwchase17/openai-functions-agent")
prompt = hub.pull("hwchase17/react")

In [58]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [59]:
prompt.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

**agent_scratchpad**:

The agent does not call a tool just once to get the desired answer, but has a structure that calls tools repeatedly until the desired answer is obtained.

Each time you call a tool, what the previous call was like, information about the previous call, and the result are stored in this field.

In [60]:
# prompt.messages

In [61]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


**Note** that we are passing in the `model`, not `model_with_tools`. That is because create_tool_calling_agent will call `.bind_tools` for us under the hood.

In [63]:
# from langchain.agents import create_tool_calling_agent

# agent = create_tool_calling_agent(llm, tools, prompt)


from langchain.agents import create_react_agent
agent = create_react_agent(llm, tools, prompt) # agent has LLM, instructions and tools it can use.

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools).

In [64]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools)

In [65]:
agent_executor.invoke({"input": "What is Mamba Mentality?"})

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py

{'input': 'What is Mamba Mentality?',
 'output': 'The Mamba Mentality is a mindset or approach to life and goals that is about being on a constant quest to try to be the best version of yourself. It consists of 5 pillars: Passion, Obsession, Relentlessness, Resiliency, and Fearlessness. It was coined by Kobe Bryant and is described in his book "The Mamba Mentality: How I Play."'}

# Custom Tools

Besides the actual function that is called, the Tool consists of several components:

1. name (str), is required and must be unique within a set of tools provided to an agent
2. description (str), is optional but recommended, as it is used by an agent to determine tool use
3. args_schema (Pydantic BaseModel), is optional but recommended, can be used to provide more information (e.g., few-shot examples) or validation for expected parameters.

There are 3 ways

[Custom Tools](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)

1. @tool decorator:

  This `@tool` decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

2. Subclass BaseTool: [Personal Favourite]
  
  You can also explicitly define a custom tool by subclassing the BaseTool class. This provides maximal control over the tool definition, but is a bit more work.

3. StructuredTool dataclass:

  You can also use a StructuredTool dataclass. This methods is a mix between the previous two. It's more convenient than inheriting from the BaseTool class, but provides more functionality than just using a decorator.

In [67]:
# Import things that are needed generically
from pydantic import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

In [68]:
@tool
def add_tool(num_1: int, num_2: int) -> int:
    """Add two numbers."""
    return num_1 + num_2



In [69]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [70]:
tools = [search, wikipedia, add_tool, multiply]

from langchain.agents import create_react_agent
agent = create_react_agent(llm, tools, prompt) # agent has LLM, instructions and tools it can use.


from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [71]:
agent_executor.invoke({"input": "What is Mamba Mentality?"})

/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(
/usr/local/lib/python3.10/dist-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: 'api' backend is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


{'input': 'What is Mamba Mentality?',
 'output': 'The Mamba Mentality is a mindset that inspires people to work hard, focus on the process, and strive for greatness, coined by Kobe Bryant to describe his constant quest to be the best version of himself.'}

In [77]:
agent_executor.invoke({"input": "Who is MK Gandhi?"})

{'input': 'Who is MK Gandhi?',
 'output': "Mohandas Karamchand Gandhi, commonly known as Mahatma Gandhi, was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule."}

In [79]:
agent_executor.invoke({"input": "What is the paper attention is all you need about?"})

{'input': 'What is the paper attention is all you need about?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

# Subclass BaseTool

In [122]:
from langchain_core.tools import BaseTool
from pydantic import BaseModel, Field
from math import pi, sqrt, cos, sin
from typing import Union, Optional, Type
import re

Pydantic will attempt to 'match' any of the types defined under Union and will use the first one that matches.[...]

As such, it is recommended that, when defining Union annotations, the most specific type is included first and followed by less specific types.

[Union](https://docs.pydantic.dev/latest/concepts/types/#unions)

In [133]:
class CircumferenceTool(BaseTool):
  name: str = "Circumference Calculator"
  description: str = " Useful when you need to calculate circumference of a circle using the radius."

  def _run(self, radius: Union[str, int, float]):
    # Extract the number (including decimals)
    number = re.findall(r"[-+]?\d*\.\d+|\d+", radius)

    # Convert to float or int
    if number:
      extracted_radius = float(number[0])
      return extracted_radius*2.0*pi

  def _arun(self, radius: Union[int, float]):
    raise NotImplementedError("This Tool Doesnot support async")

In [134]:
desc = (
    "Useful when you need to calculate the length of an hypotenuse " +
    "given one or two sides of a triangle and/or an angle (in degrees). " +
    "To use this tool, there must be atleast two of the following parameters:" +
    "['adjacent_side', 'opposite_side', 'angle']"
)

In [135]:
class PythagorasTool(BaseTool):
  name: str = "Hypotenuse Calculator"
  description: str = desc

  def _run(
      self,
      adjacent_side: Optional[Union[int, float]] = None,
      opposite_side: Optional[Union[int, float]] = None,
      angle: Optional[Union[int, float]] = None
  ):
    # check for the values we have been given
    if adjacent_side and opposite_side:
        return sqrt(float(adjacent_side)**2 + float(opposite_side)**2)

    elif adjacent_side and angle:
        return adjacent_side / cos(float(angle))

    elif opposite_side and angle:
        return opposite_side / sin(float(angle))

    else:
        return "Could not calculate the hypotenuse of the triangle. Need two or more of `adjacent_side`, `opposite_side`, or `angle`."

  def _arun(self, query:str):
    raise NotImplementedError("This tool doesnot support async")

In [136]:
tools = [search, wikipedia]

In [137]:
tools.append(CircumferenceTool())
tools.append(PythagorasTool())

In [138]:
print(tools)

[DuckDuckGoSearchRun(api_wrapper=DuckDuckGoSearchAPIWrapper(region='wt-wt', safesearch='moderate', time='y', max_results=5, backend='api', source='text')), WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)), CircumferenceTool(), PythagorasTool()]


In [139]:
# tools = [search, wikipedia, CircumferenceTool(), PythagorasTool()]

In [140]:
from langchain.agents import create_react_agent
agent = create_react_agent(llm, tools, prompt) # agent has LLM, instructions and tools it can use.


from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [131]:
agent_executor.invoke({"input": "If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?"})

{'input': 'If I have a triangle with two sides of length 51cm and 34cm, what is the length of the hypotenuse?',
 'output': 'Agent stopped due to iteration limit or time limit.'}

In [142]:
agent_executor.invoke({"input": "What is the circumference of a circle that has a radius of 7.81mm?"})

{'input': 'What is the circumference of a circle that has a radius of 7.81mm?',
 'output': '49.071677249072565'}